In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
import os
from torchvision import datasets, transforms
from torch import optim, nn, unsqueeze
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split



In [2]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset0 = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=False, transform=transform)

# Also create a validation set 
trainset, valset = train_test_split(trainset0, test_size=10000, random_state=42)
#pytorch alternative for spliting into train and validation sets
#https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

batch_size = 16

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

In [3]:
training_data = enumerate(trainloader)
batch_idx, (images, labels) = next(training_data)
print(type(images)) # Checking the datatype 
print(images.shape) # the size of the image
print(labels.shape) # the size of the labels

#64: Represents 64 images
# 1 : One color channel ==>> Grayscale
# 28 by 28 pixel: the shape of these images so we can visualize it.

<class 'torch.Tensor'>
torch.Size([16, 1, 28, 28])
torch.Size([16])


In [30]:
class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()

        self.conv_neural_network_layers = nn.Sequential(
                # output_sizeOfEachConvLayer = [(in_channel + 2*padding - kernel_size) / stride] + 1

                # We have in_channels=1 because our input is a grayscale image
                nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1, stride=1), # (N, 1, 28, 28) 
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2), 

                # output of second conv layer
                nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, stride=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2), 

                # output of third conv layer
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2) )

        # Linear layer
        self.linear_layers = nn.Sequential(
                nn.Linear(64*3*3, 10))
                # nn.Linear(16, 10)) # The output is 10 which should match the size of our class


    # Defining the forward pass 
    def forward(self, x):
        x = self.conv_neural_network_layers(x)
        # After we get the output of our convolutional layer we must flatten it or rearrange the output into a vector
        x = torch.flatten(x, 1) # same as x = x.view(x.size(0), -1)
        # Then pass it through the linear layer
        x = self.linear_layers(x)

        return x

model = Network()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


loss_fn = nn.BCEWithLogitsLoss() #nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y_hot = F.one_hot(y, 10)
        y_hot = torch.zeros(batch_size, 10)
        y_hot[range(y_hot.shape[0]), y]=1      

        X, y_hot = X.to(device), y_hot.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y_hot)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#We also check the model’s performance against the test dataset to ensure it is learning.

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        y_hot = F.one_hot(y, 10)
        y_hot = torch.zeros(batch_size, 10)
        y_hot[range(y_hot.shape[0]), y]=1      

        X, y_hot = X.to(device), y_hot.to(device)

        # Compute prediction error
        pred = model(X)
        test_loss += loss_fn(pred, y_hot).item()
        correct += (pred.argmax(axis=1) == y_hot.argmax(axis=1)).type(torch.float).sum().item()

    test_loss /= 10000
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


## Experiments

### Epochs

In [32]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(valloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fdb3a972310>
Traceback (most recent call last):
  File "/Users/brunaguedes/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/Users/brunaguedes/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1295, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Test Error: 
 Accuracy: 98.4%, Avg loss: 0.000798 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.000526 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 98.9%, Avg loss: 0.000572 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.000552 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.000513 

Done!


In [33]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(valloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.000533 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 99.3%, Avg loss: 0.000476 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.000584 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.000547 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.000596 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.000586 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.000774 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.000723 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.000676 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.000789 

Done!


### Learning rate

In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(valloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
Test Error: 
 Accuracy: 97.2%, Avg loss: 0.001515 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.001641 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 97.7%, Avg loss: 0.001346 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 97.6%, Avg loss: 0.001296 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 97.4%, Avg loss: 0.001542 

Done!


In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(valloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 98.2%, Avg loss: 0.000985 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 98.3%, Avg loss: 0.000930 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 98.4%, Avg loss: 0.000893 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 98.5%, Avg loss: 0.000872 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 98.5%, Avg loss: 0.000855 

Done!


In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(valloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000854 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000853 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000852 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000850 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.000850 

Done!


### Mini-batch

In [45]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset0 = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=False, transform=transform)

# Also create a validation set 
trainset, valset = train_test_split(trainset0, test_size=10000, random_state=42)
#pytorch alternative for spliting into train and validation sets
#https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

batch_size = 16

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

In [38]:
# batch_size = 8
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(valloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.001697 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.001694 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.001691 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.001689 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.001686 

Done!


In [40]:
# batch_size = 4
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(valloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.003367 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.003361 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.003358 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.003352 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.003348 

Done!


In [44]:
# batch_size = 2
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(valloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.006675 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.006665 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.006661 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.006654 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.006651 

Done!


## Now we choose final model, run the test set!
Choice: epochs = 5, batch_size = 16, learning_rate = 1e-3

In [46]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.000703 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 98.9%, Avg loss: 0.000691 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 98.9%, Avg loss: 0.000732 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 98.9%, Avg loss: 0.000774 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 98.9%, Avg loss: 0.000786 

Done!


# Q9

Question 9: Add some data augmentations to the data loader for the training set. Why do we only augment the training data? Play around with the augmentations available in torchvision. Try to get better performance than the baseline. Once you are happy with your choice of augmentations, run both the baseline and the augmented version on the test set and report the accuracies in your report.

Notes:
- When we use data augmentation, loading the whole data into memory creates a problem. If we use the dataloader, then the data augmentation is randomly applied each time. If we preload the data the dataloader only applies sthe data augmentation once, defeating the purpose. Here, we'll need to use pytorch's random split function instead:
https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split

- Apply this to the dataset object before passing it to the dataloader.

In [47]:
# Define a transform to normalize the data (previosly used and now only for val and test)
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,))])

# with data augmentation for training only
train_transform = transforms.Compose(
                    [transforms.ToPILImage(),
                    # transforms.RandomRotation(30),
                    transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                    transforms.ColorJitter(brightness=0.2, contrast=0.2),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset0 = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=True, transform=train_transform)
testset = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=False, transform=transform)

# Also create a validation set 
trainset, valset = train_test_split(trainset0, test_size=10000, random_state=42)
#pytorch alternative for spliting into train and validation sets
#https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

batch_size = 4

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)

TypeError: pic should be Tensor or ndarray. Got <class 'PIL.Image.Image'>.